In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os

os.environ["openai_api_key"] = "ENTER OPENAI API KEY"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = "ENTER SERPER API KEY"

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of your company's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [8]:
from crewai_tools import BaseTool

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2
)

In [15]:
inputs = {
    "lead_name": input("Company:"),
    "industry": input("Industry:"),
    "key_decision_maker": input("Decision Maker:"),
    "position": input("Position:"),
    "milestone": input("milestone:")
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Synergy, a company in the Software in Maritime sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information on Synergy, including their background, key personnel, recent milestones, and identified needs to provide a comprehensive report and suggest personalized engagement strategies.

Action: Search the internet
Action Input: {"search_query": "Synergy Software in Maritime company profile"} 


Search results: Title: Synergy Marine Group: Ship Management Company
Link: https://www.synergymarinegr

In [16]:
from IPython.display import Markdown
Markdown(result)

Subject: Enhancing Synergy's Ship Management Operations with Innovative Solutions

Dear Captain,

I hope this message finds you well. I am reaching out to you on behalf of our team at [Your Company Name] with an exciting opportunity to optimize Synergy's fleet efficiency and enhance productivity through our innovative solutions.

Having recently expanded your services to include advanced technology solutions for ship technical management, it is evident that Synergy is committed to embracing innovation and efficiency. Our tailored services align perfectly with your goals of streamlining ship management operations using data-driven decision-making and forecasting.

At [Your Company Name], we specialize in providing advanced technology tools for ship technical management, data analytics for optimized decision-making, and streamlined operations for enhanced efficiency and productivity. Our solutions have a proven track record of helping companies in the maritime sector achieve their operational objectives effectively.

We understand the importance of aligning with Synergy's company culture and values, and we are dedicated to supporting your business needs by offering personalized solutions that cater to your specific requirements. By leveraging our expertise and technology, we can assist Synergy in achieving even greater success in the maritime industry.

I would love the opportunity to discuss how our solutions can benefit Synergy further. Please let me know a convenient time for us to connect and explore the possibilities of a successful partnership.

Thank you for your time, Captain. I look forward to the opportunity to collaborate with Synergy and contribute to your continued success.

Warm regards,

[Your Name]
Lead Sales Representative
[Your Company Name]
[Contact Information]